In [1]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime,timedelta
warnings.filterwarnings("ignore")

In [17]:
## PROBLEM
    ## It takes in every day up until the last one. Need to create a new team list using the final day in the "Up_To_Today.csv" file
    ## Then loop through that team list
def rolling_windows(window):
    bat_data = pd.read_csv("Clean_Bat.csv",index_col = "Unnamed: 0")
    pitch_data = pd.read_csv("Clean_Pitch.csv",index_col = "Unnamed: 0")
    prior_games = pd.read_csv("Previous_Games.csv",index_col="Unnamed: 0")
    total_rolling_bat = pd.DataFrame()
    post_start = prior_games[prior_games['Date'] > prior_games.Date.unique()[window]]
    season_2022 = post_start[post_start['Date'] < "2023-03-30"]
    date_after_window = datetime.strptime("2023-03-30","%Y-%m-%d") + timedelta(days = window)
    date_after_window = date_after_window.strftime("%Y-%m-%d")
    season_2023 = post_start[post_start['Date'] > date_after_window]
    total_clean_date = pd.concat([season_2022,season_2023])
    next_day = datetime.strptime(total_clean_date.iloc[-1]['Date'],"%Y-%m-%d") + timedelta(days = 1)
    next_day = next_day.strftime("%Y-%m-%d")
    date_list = list(total_clean_date.Date.unique())
    date_list.append(next_day)
    for date in date_list:
        date_df = bat_data[bat_data['Date'] < date]
        past = datetime.strptime(date,"%Y-%m-%d") - timedelta(days = window)
        past = past.strftime("%Y-%m-%d")
        date_df = date_df[date_df['Date'] >= past]
        today = bat_data[bat_data['Date'] == date]
        for team in today.Team.unique():
            team_df = date_df.groupby("Team").get_group(team)
            temp_df = pd.DataFrame(team_df[['PA','AB','R','H','2B','3B','HR','RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']].sum()).T
            temp_df = temp_df.add_suffix("_Bat")
            temp_df['Team'] = team
            temp_df['Date'] = date
            temp_df['Games'] = len(team_df.Date.unique())
            total_rolling_bat = pd.concat([total_rolling_bat,temp_df])
    total_rolling_pitch = pd.DataFrame()
    for date in date_list:
        date_df = pitch_data[pitch_data['Date'] < date]
        past = datetime.strptime(date,"%Y-%m-%d") - timedelta(days = window)
        past = past.strftime("%Y-%m-%d")
        date_df = date_df[date_df['Date'] >= past]
        today = pitch_data[pitch_data['Date'] == date]
        for team in today.Team.unique():
            team_df = date_df.groupby("Team").get_group(team)
            for x in range(0,8):
                team_df['IP'] = team_df['IP'].replace(x+.1,x+.33)
                team_df['IP'] = team_df['IP'].replace(x+.2,x+.67)
            temp_df = pd.DataFrame(team_df[['IP','H','R','ER','BB','SO','HR','HBP','AB','2B','3B','IBB','SB','CS','PO','BF','GB/FB']].sum()).T
            temp_df = temp_df.add_suffix("_Pitch")
            temp_df['Team'] = team
            temp_df['Date'] = date
            total_rolling_pitch = pd.concat([total_rolling_pitch,temp_df])
    ip_list = []
    for ip in total_rolling_pitch['IP_Pitch']:
        if float("." + str(ip).split(".")[1]) > 0.5 and float("." + str(ip).split(".")[1]) < 0.835:
            ip = int(str(ip).split(".")[0]) + 0.67
        elif float("." + str(ip).split(".")[1]) < 0.5 and float("." + str(ip).split(".")[1]) > 0.165:
            ip = int(str(ip).split(".")[0]) + 0.33
        else:
            ip = int(ip)
        ip_list.append(ip)
    total_rolling_pitch['IP_Pitch'] = ip_list
    final_df = pd.DataFrame()
    for date in total_rolling_bat.Date.unique():
        pitch_df = total_rolling_pitch[total_rolling_pitch['Date'] == date]
        bat_df = total_rolling_bat[total_rolling_bat['Date'] == date]
        for team in pitch_df.Team.unique():
            team_pitch = pitch_df[pitch_df['Team'] == team]
            team_bat = bat_df[bat_df['Team'] == team]
            team_pitch = team_pitch.set_index("Team")
            team_bat.drop(columns = 'Date',inplace = True)
            team_bat.set_index("Team",inplace = True)
            team = pd.concat([team_pitch,team_bat],axis = 1)
            team.reset_index(inplace=True)
            team.set_index("Date",inplace=True)
            team.reset_index(inplace=True)
            final_df = pd.concat([final_df,team])
    final_df.to_csv(str(window) + "_Day_Rolling_Window.csv")
    print("Finished running with window of " + str(window) + " days.")
    return

In [18]:
rolling_windows(7)
rolling_windows(10)
rolling_windows(15)

2023-07-08


KeyboardInterrupt: 